In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from astropy.constants import m_p, M_sun
import astropy.units as u

sys.path.append('../pipeline_scripts/')
from pipeline_main_nosink import pipeline_nosink
from pipeline_2D_nosink import to_osyris_ivs
from pipeline_main import sink_positions, serialize_directory, sinks, true_sinks

core_data = {}
initial_sn = [158, 222, 176, 212, 235, 341, 401, 403, 445]

### - Mass
### - Mass weighted average velocity
### - RMS Velcotiy (also mass weighted)
### - average magnetic field strength (volume weighted)
### - rms fluctuating magnetic field strenght $$
### - Net kinetic energy, magnetic energy, thermal energy, net angular moment

## $$E_\text{tot} = \rho\epsilon + \frac{1}{2}\rho v^2+\frac{B^2}{8\pi}$$

### $$\langle V_\text{core}\rangle= \frac{\sum\;v_{c}m_{c}}{\sum m_{c}}$$
### $$v_\text{rel} = (v_{c} - \langle V_\text{core}\rangle)$$
### $$V_\text{RMS} = \sqrt{\frac{\sum\;v_\text{rel}^2m_{c}}{\sum m_{c}}}$$

### $$\langle B_\text{core}\rangle= \frac{\sum\;B_{c}vol_{c}}{\sum vol_{c}}$$
### $$B_\text{rel} = (B_{c} - \langle B_\text{core}\rangle)$$
### $$B_\text{RMS} = \sqrt{\frac{\sum\;B_\text{rel}^2vol_{c}}{\sum vol_{c}}}$$

### $$E_\text{kin} = \frac{1}{2}\sum m_c \cdot v_\text{rel}^2$$
### $$E_\text{mag}  = \frac{1}{8\pi}\sum B_c^2$$
### $$E_\text{therm} = \sum P_{gas, c}\cdot vol_c$$
### $$E_\text{rot} = \sum\frac{1}{2|r|^4}mr^2 (r\times v_\text{rel})^2$$

In [4]:
data = pipeline_nosink(snap = initial_sn[0], run = f'sink{6}fmr01', sink_pos = sink_positions[6])

Initialising patch data


100%|██████████| 4756/4756 [00:23<00:00, 205.70it/s]


In [168]:
data_14 = pipeline_nosink(snap = initial_sn[3], run = f'sink{14}fmr01', sink_pos = sink_positions[14])

Initialising patch data


100%|██████████| 4715/4715 [00:35<00:00, 133.66it/s]


In [246]:
variables = ['d', 'vol', 'P']
vectors = ['v', 'B', 'rel_xyz']
core_characteristics = {sink: {} for sink in sinks}
radius_au = 1e4 


for i, sink in enumerate(sinks):
    data = pipeline_nosink(snap = initial_sn[i], run = f'sink{sink}fmr01', sink_pos = sink_positions[sink])
    patch_values = {key: [] for key in variables}
    patch_vectors = {key: [] for key in vectors}
    radius_code = radius_au / data.au_length

    pp = [p for p in data.sn.patches if (p.dist_xyz < radius_code).any() and p.level > 5]
    w= np.array([p.level for p in pp]).argsort()[::-1]
    sorted_patches = [pp[w[i]] for i in range(len(pp))]

        
    for p in tqdm.tqdm(sorted_patches, disable = not data.loading_bar):
            
            p.v = np.concatenate([p.var(f'u'+axis)[None,...] for axis in ['x','y','z']], axis = 0)
            p.B = np.concatenate([p.var(f'b'+axis)[None,...] for axis in ['x','y','z']], axis = 0)
            p.vol = np.prod(p.ds) * np.ones((16,16,16))
            
            nbors = [data.sn.patchid[i] for i in p.nbor_ids if i in data.sn.patchid]
            children = [ n for n in nbors if n.level == p.level + 1]
            leafs = [n for n in children if ((n.position - p.position)**2).sum() < ((p.size)**2).sum()/12]
            if len(leafs) == 8: continue
            to_extract = p.dist_xyz < radius_code
            for lp in leafs:
                leaf_extent = np.vstack((lp.position - 0.5 * lp.size, lp.position + 0.5 * lp.size)).T
                covered_bool = ~np.all((p.xyz > leaf_extent[:, 0, None, None, None]) & (p.xyz < leaf_extent[:, 1, None, None, None]), axis=0)
                to_extract *= covered_bool
            for i, ivs in enumerate(variables):
                if hasattr(p, ivs):
                    new_value = getattr(p, ivs)[to_extract].T               
                else:
                    new_value = p.var(ivs)[to_extract].T
                patch_values[ivs].extend(new_value.tolist())

            for i, ivs in enumerate(vectors):
                new_vector = getattr(p, ivs)[:, to_extract].T
                patch_vectors[ivs].extend(new_vector.tolist())

    for key in patch_values:
        patch_values[key] = np.array(patch_values[key])
    for key in patch_vectors:
        patch_vectors[key] = np.array(patch_vectors[key])

    cell_mass = patch_values['d'] * patch_values['vol']
    V_core = np.sum(patch_vectors['v'] * cell_mass[:,None], axis = 0)  / cell_mass.sum()
    V_rms = (np.sum(np.linalg.norm(patch_vectors['v'] - V_core, axis = 1)**2 * cell_mass) / cell_mass.sum())**0.5

    B_core = np.sum(patch_vectors['B'] * patch_values['vol'][:,None], axis = 0)  / patch_values['vol'].sum()
    B_rms = (np.sum(np.linalg.norm(patch_vectors['B'] - B_core, axis = 1)**2 * patch_values['vol']) / patch_values['vol'].sum())**0.5

    E_kin = (0.5 * cell_mass * np.linalg.norm(patch_vectors['v'] - V_core, axis = 1)**2).sum(); 
    E_mag = np.sum(np.linalg.norm(patch_vectors['B'], axis = 1)**2 * patch_values['vol'] / (8 * np.pi))
    E_therm = np.sum(patch_values['P'] * patch_values['vol'])
    E_rot = np.sum(0.5 * np.linalg.norm(patch_vectors['rel_xyz'], axis = 1)**(-2) * cell_mass 
                * np.linalg.norm(np.cross(patch_vectors['rel_xyz'], patch_vectors['v'] - V_core), axis = 1)**2)

    core_characteristics[sink]['Mcore'] = cell_mass.sum() * data.msun_mass
    core_characteristics[sink]['V_core'] = V_core * data.cms_velocity * 1e-5
    core_characteristics[sink]['V_rms'] = V_rms * data.cms_velocity * 1e-5

    core_characteristics[sink]['B_core'] = B_core * data.sn.scaling.b
    core_characteristics[sink]['B_rms'] = B_rms * data.sn.scaling.b

    core_characteristics[sink]['E_kin'] = E_kin * data.sn.scaling.e
    core_characteristics[sink]['E_mag'] = E_mag * data.sn.scaling.e
    core_characteristics[sink]['E_therm'] = E_therm * data.sn.scaling.e
    core_characteristics[sink]['E_rot'] = E_rot * data.sn.scaling.e

#serialize_directory(filename = 'core_data.pkl', directory = core_characteristics, store = True, folder = '/groups/astro/kxm508/codes/python_dispatch/validations/')

Initialising patch data


100%|██████████| 3043/3043 [01:03<00:00, 47.95it/s] 


Initialising patch data


100%|██████████| 2400/2400 [00:56<00:00, 42.79it/s] 


Initialising patch data


...

100%|██████████| 2067/2067 [00:34<00:00, 60.19it/s] 


Initialising patch data


100%|██████████| 3415/3415 [01:08<00:00, 50.07it/s] 


Initialising patch data


100%|██████████| 3029/3029 [00:57<00:00, 52.56it/s] 


In [2]:
core_characteristics = serialize_directory(filename = 'core_data.pkl', store = False, folder = '/groups/astro/kxm508/codes/python_dispatch/validations/')

In [20]:
sink = 225
for key in core_characteristics[sink].keys():
    print(key, core_characteristics[sink][key])

print('n', f"{(core_characteristics[sink]['Mcore'] * M_sun / (4/3 * (1e4 * data.sn.cgs.au * 1e-5 * u.km)**3 *np.pi) / (m_p * 2.34)).to('1/cm3').value:1.2e}")
print('Avg_B', np.linalg.norm(core_characteristics[sink]['B_core'] * 1e6))
print('B_rms', core_characteristics[sink]['B_rms'] * 1e6)


Mcore 1.7082077414335544
V_core [ 0.01384038 -0.0883597   0.16794702]
V_rms 0.3465529695888759
B_core [ 1.33795578e-05 -3.45513842e-05 -5.03957415e-05]
B_rms 5.6814873520170404e-05
E_kin 2.0402562349599195e+42
E_mag 3.172532885480668e+41
E_therm 1.1531325602497829e+42
E_rot 1.0415286594190174e+42
n 6.19e+04
Avg_B 62.55031157290253
B_rms 56.8148735201704
